In [2]:
engine = sqlalchemy.create_engine('mysql+pymysql://root:@localhost:3306/samsung_scrape')

In [2]:
df = pd.read_excel(r'C:\Users\Anurag\Downloads\url_data.xlsx')
df

,Platform,Category,ID,url
0,Amazon,TV,1,https://www.amazon.in/Samsung-inches-Wondertai...
1,Flipkart,TV,2,https://www.flipkart.com/samsung-frame-138cm-5...
2,VS,TV,3,https://www.vijaysales.com/Samsung-QA65Q70R-65...
3,VS,TV,4,https://www.vijaysales.com/Samsung-UA43N5470-4...
4,VS,TV,5,https://www.vijaysales.com/Samsung-UA65RU7470-...


In [5]:
df = pd.read_sql_table('url_read', con = engine)
df

,Platform,Category,ID,URL
0,Amazon,TV,1,https://www.amazon.in/Samsung-inches-Wondertai...
1,Flipkart,TV,2,https://www.flipkart.com/samsung-frame-138cm-5...
2,VS,TV,3,https://www.vijaysales.com/Samsung-QA65Q70R-65...
3,VS,TV,4,https://www.vijaysales.com/Samsung-UA43N5470-4...
4,VS,TV,5,https://www.vijaysales.com/Samsung-UA65RU7470-...


In [8]:
df['URL'][0]

'https://www.amazon.in/Samsung-inches-Wondertainment-Ready-UA32TE40AAKXXL/dp/B086RB1XCK/ref=sr_1_1_sspa?dchild=1&keywords=Samsung+LED&qid=1593759002&sr=8-1-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUExOThIWklVQjkwTldJJmVuY3J5cHRlZElkPUEwNDA0NTI2UVY5SVg0V1ZNTlFNJmVuY3J5cHRlZEFkSWQ9QTA5MzU5MDZFMDBHUjI0UzNUSTYmd2lkZ2V0TmFtZT1zcF9hdGYmYWN0aW9uPWNsaWNrUmVkaXJlY3QmZG9Ob3RMb2dDbGljaz10cnVl'

In [9]:
Query = """
        SELECT url from url_read where Platform = 'VS'
        """
df1 = pd.read_sql_query(Query, con = engine)
df1

,url
0,https://www.vijaysales.com/Samsung-QA65Q70R-65...
1,https://www.vijaysales.com/Samsung-UA43N5470-4...
2,https://www.vijaysales.com/Samsung-UA65RU7470-...


In [10]:
for i in df1['url'].values:
    print(i)

https://www.vijaysales.com/Samsung-QA65Q70R-65-163cm-4K-Ultra-HD-Smart-QLED-TV/10769
https://www.vijaysales.com/Samsung-UA43N5470-43-108-cm-Full-HD-Smart-LED-TV/10489
https://www.vijaysales.com/Samsung-UA65RU7470-65-163cm-4K-Ultra-HD-Smart-LED-TV/10779


In [22]:
from datetime import date,datetime
from selenium import webdriver
import pandas as pd
from time import sleep
import sqlalchemy
import pandas as pd
import pymysql
import time

engine = sqlalchemy.create_engine('mysql+pymysql://root:@localhost:3306/samsung_scrape')

def VS(engine):
    Query = """
            SELECT * from url_read where Platform = 'VS'
            """
    df = pd.read_sql_query(Query, con = engine)
    
    
    for i in df.itertuples():
        driver = webdriver.Chrome(executable_path=r"C:\Users\Anurag\Downloads\chromedriver_win32\chromedriver.exe")
        plat_, cate_, id_, name, o_rating, d_price, dis, mrp, stock, date, time_, seller = [],[],[],[],[],[],[],[],[],[],[],[]
        plat_.append(i[1])
        cate_.append(i[2])
        id_.append(i[3])
        seller.append(i[1])
        # Item-details 
        driver.get(i[4])
        time.sleep(3)
        
        try:
            path = '/html/body/form/div[4]/div[16]/div[4]/div/div[2]/div[2]/div[1]/h1'
            name.append(driver.find_element_by_xpath(path).text)
        except:
            name.append('Not Available')
            pass
        try:
            mrp.append(float(driver.find_element_by_xpath('/html/body/form/div[4]/div[16]/div[4]/div/div[2]/div[2]/div[3]/div[1]/span[2]').text)) # MRP
        except:
            mrp.append('Not Available')
        try:
            d_price.append(float(driver.find_element_by_xpath('/html/body/form/div[4]/div[16]/div[4]/div/div[2]/div[2]/div[3]/div[2]/div[1]/div[1]/span[2]').text)) # Seller price
        except:
            d_price.append('Not Available')
        try:
            dis.append(driver.find_element_by_xpath('/html/body/form/div[4]/div[16]/div[4]/div/div[2]/div[2]/div[3]/div[2]/div[1]/div[2]').text) # Discount
        except:
            dis.append('Not Available')
        # Stock
        try:
            if driver.find_element_by_xpath("//span[@class = 'title colstockqthcol avlqty']").is_displayed:
                stock.append("Low Stock Avaiable")
        except:
            pass
        try:
            if driver.find_element_by_xpath("//div[@class=  'clssoldout_Title']").is_displayed:
                stock.append("NO Stock Avaiable")
        except:
            stock.append("Stock Avaiable")
        date.append(datetime.now().strftime("%d-%m-%Y")) 
        time_.append(datetime.now().strftime("%H:00 %p"))
        
    df2 = pd.DataFrame({'Date':date,'Time':time_,'Product_Category': cate_,'Product_platform':plat_,'Product_name': name,'Product_id': id_, 'Product_MRP': mrp,
                   'Product_Selling_Price': d_price,'Product_discount': dis,'Product_availability': stock, 'Seller':seller})
    df2.to_sql('Scrapped_data_all',con=engine,if_exists='append',index=False)

In [23]:
VS(engine)

,Date,Product_Category,Product_MRP,Product_Selling_Price,Product_availability,Product_discount,Product_id,Product_name,Product_platform,Seller,Time
0,06-07-2020,TV,"₹ 1,89,900","₹ 92,590",NO Stock Avaiable,51% OFF,5,Samsung UA65RU7470 65 (163cm) 4K Ultra HD Smar...,VS,VS,00:00 AM


In [ ]:
## OUT OF STOCK IS WRONG

In [3]:
df

,Platform,Category,ID,url
0,Amazon,TV,1,https://www.amazon.in/Samsung-inches-Wondertai...
1,Flipkart,TV,2,https://www.flipkart.com/samsung-frame-138cm-5...
2,VS,TV,3,https://www.vijaysales.com/Samsung-QA65Q70R-65...
3,VS,TV,4,https://www.vijaysales.com/Samsung-UA43N5470-4...
4,VS,TV,5,https://www.vijaysales.com/Samsung-UA65RU7470-...


In [10]:
for i in df.itertuples():
    plat_ = i[1]
    cat_ = i[2]
    id_ = i[3]
    url_ = i[4]
    print(plat_, cat_, id_, url_)


Amazon TV 1 https://www.amazon.in/Samsung-inches-Wondertainment-Ready-UA32TE40AAKXXL/dp/B086RB1XCK/ref=sr_1_1_sspa?dchild=1&keywords=Samsung+LED&qid=1593759002&sr=8-1-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUExOThIWklVQjkwTldJJmVuY3J5cHRlZElkPUEwNDA0NTI2UVY5SVg0V1ZNTlFNJmVuY3J5cHRlZEFkSWQ9QTA5MzU5MDZFMDBHUjI0UzNUSTYmd2lkZ2V0TmFtZT1zcF9hdGYmYWN0aW9uPWNsaWNrUmVkaXJlY3QmZG9Ob3RMb2dDbGljaz10cnVl


In [11]:
###################################

In [ ]:
# admin
# Super.com1
# fist page last 4

## Croma

In [15]:
from datetime import date,datetime
from selenium import webdriver
import pandas as pd
from time import sleep
import sqlalchemy
import pandas as pd
import pymysql
import time

engine = sqlalchemy.create_engine('mysql+pymysql://root:@localhost:3306/samsung_scrape')

def croma(engine):
    # Read the url = Croma from url_read table
    Query = """
            SELECT * from url_read where Platform = 'Croma'
            """
    df = pd.read_sql_query(Query, con = engine)
    
    
    for i in df.itertuples():
        driver = webdriver.Chrome(executable_path=r"C:\Users\Anurag\Downloads\chromedriver_win32\chromedriver.exe")
        plat_, cate_, id_, name, o_rating, d_price, dis, mrp, stock, date, time_, seller = [],[],[],[],[],[],[],[],[],[],[],[]
        plat_.append(i[1])
        cate_.append(i[2])
        id_.append(i[3])
        seller.append(i[1])
        # Item-details 
        driver.get(i[4])
        time.sleep(3)
        
        try:
            name.append(driver.find_element_by_xpath("//small[@class = 'product_name_small']").text)
        except:
            name.append('Not Available')
        try:
            o_rating.append(driver.find_element_by_xpath("//div[@class = 'star-rating-txt']").text) # Rating
        except:
            o_rating.append('Not Available')
        try:
            d_price.append(float(driver.find_element_by_xpath("//span[@class = 'pdpPrice']").text.split('₹')[1])) # d_price = ₹75599
        except:
            d_price.append('Not Available')
        try:
            dis.append(driver.find_element_by_xpath("//div[@class = 'pdpPriceDiscount']").text.split()[0]) # 19% OFF
        except:
            dis.append('Not Available')
        try:
            mrp.append(float(driver.find_element_by_xpath("//span[@class = 'pdpPriceMrp']").text.split('₹')[1])) # MRP
        except:
            mrp.append('Not Available')
        # Stock
        stock.append('Not Available')
        date.append(datetime.now().strftime("%d-%m-%Y")) 
        time_.append(datetime.now().strftime("%H:00 %p"))
        
    df2 = pd.DataFrame({'Date':date,'Time':time_,'Product_Category': cate_,'Product_platform':plat_,'Product_name': name,'Product_id': id_, 'Product_MRP': mrp,
                   'Product_Selling_Price': d_price,'Product_discount': dis,'Product_availability': stock, 'Seller':seller})
    df2.to_sql('Scrapped_data_all',con=engine,if_exists='append',index=False)
    # return(df2)

In [16]:
croma(engine)

(['Croma'],
 ['TV'],
 ['32N400'],
 ['Not Available'],
 ['Not Available'],
 ['Not Available'],
 ['Not Available'],
 ['Not Available'],
 ['Not Available'],
 ['05-07-2020'],
 ['23:00 PM'],
 ['Croma'])

In [4]:
# Read the url = Croma from url_read table
Query = """
        SELECT * from url_read where Platform = 'Croma'
        """

# Read table with a query
df = pd.read_sql_query(Query, con = engine)
df

,Platform,Category,ID,URL
0,Croma,MW,MC32K7056CB/TL,https://www.croma.com/samsung-32-litres-convec...
1,Croma,TV,32N400,https://www.croma.com/samsung-81-cm-32-inch-hd...


In [5]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from datetime import datetime
import numpy as np
from selenium.common.exceptions import NoSuchElementException

for i in df.itertuples():
    driver = webdriver.Chrome(executable_path=r"C:\Users\Anurag\Downloads\chromedriver_win32\chromedriver.exe")
    plat_, cate_, id_, name, o_rating, d_price, dis, mrp, stock, date, time_, seller = [],[],[],[],[],[],[],[],[],[],[],[]
    plat_.append(i[1])
    cate_.append(i[2])
    id_.append(i[3])
    seller.append(i[1])
    # Item-details # At here difine diff-diff function or module of site which return df a/c plateform:- import paytm
    driver.get(i[4])
    time.sleep(3)
    try:
        name.append(driver.find_element_by_xpath("//small[@class = 'product_name_small']").text)
    except:
        name.append('Not Available')
    try:
        o_rating.append(driver.find_element_by_xpath("//div[@class = 'star-rating-txt']").text) # Rating
    except:
        o_rating.append('Not Available')
    try:
        d_price.append(float(driver.find_element_by_xpath("//span[@class = 'pdpPrice']").text.split('₹')[1])) # d_price = ₹75599
    except:
        d_price.append('Not Available')
    try:
        dis.append(driver.find_element_by_xpath("//div[@class = 'pdpPriceDiscount']").text.split()[0]) # 19% OFF
    except:
        dis.append('Not Available')
    try:
        mrp.append(float(driver.find_element_by_xpath("//span[@class = 'pdpPriceMrp']").text.split('₹')[1])) # MRP
    except:
        mrp.append('Not Available')
    # Stock
    stock.append('Not Available')
    date.append(datetime.now().strftime("%d-%m-%Y")) 
    time_.append(datetime.now().strftime("%H:00 %p"))

In [6]:
df = pd.DataFrame({'Date':date,'Time':time_,'Product_Category': cate_,'Product_platform':plat_,'Product_name': name,'Product_id': id_, 'Product_MRP': mrp,
                   'Product_Selling_Price': d_price,'Product_discount': dis,'Product_availability': stock, 'Seller':seller})
df

,Date,Product_Category,Product_MRP,Product_Selling_Price,Product_availability,Product_discount,Product_id,Product_name,Product_platform,Seller,Time
0,2020-07-05,MW,"25,850.00","22,499.00",Nan,13%,MC32K7056CB/TL,Samsung 32 litres Convection Microwave Oven (M...,Croma,Croma,22:56:43
1,2020-07-05,TV,"24,900.00","17,990.00",Nan,28%,32N400,Samsung 81 cm (32 inch) HD LED Smart TV (32N42...,Croma,Croma,22:56:57


In [30]:
df.to_sql(name = 'url_data', con = engine, index = False, if_exists='append')

In [34]:
df1 = pd.read_csv(r'C:\Users\Anurag\Desktop\New folder\Scrapped_data_all.csv')
df1.columns

Index(['Date', 'Time', 'Product_Category', 'Product_platform', 'Product_name',
       'Product_id', ' Product_MRP', 'Product_Selling_Price',
       ' Product_discount', ' Seller', 'Product_availability',
       ' Price_fluctuation', ' Percentage_change'],
      dtype='object')

In [35]:
df.columns  

Index(['Date', 'Product_Category', 'Product_availability', 'Product_discount',
       'Product_discounted_price', 'Product_id', 'Product_name',
       'Product_platform', 'Product_price', 'Time'],
      dtype='object')

In [36]:
df.rename({'Product_price':' Product_MRP','Product_discounted_price': 'Product_Selling_Price'},inplace = True)
df

,Date,Product_Category,Product_availability,Product_discount,Product_discounted_price,Product_id,Product_name,Product_platform,Product_price,Time
0,2020-07-04,MW,Nan,13%,"22,499.00",MC32K7056CB/TL,Samsung 32 litres Convection Microwave Oven (M...,Croma,"25,850.00",01:14:07
1,2020-07-04,TV,Nan,28%,"17,990.00",32N400,Samsung 81 cm (32 inch) HD LED Smart TV (32N42...,Croma,"24,900.00",01:14:30


In [38]:
df.to_sql(name = 'scrape_data_all', con = engine, index = False, if_exists='append')

InternalError: (pymysql.err.InternalError) (1054, "Unknown column 'Product_discount' in 'field list'")
[SQL: INSERT INTO scrape_data_all (`Date`, `Product_Category`, `Product_availability`, `Product_discount`, `Product_discounted_price`, `Product_id`, `Product_name`, `Product_platform`, `Product_price`, `Time`) VALUES (%(Date)s, %(Product_Category)s, %(Product_availability)s, %(Product_discount)s, %(Product_discounted_price)s, %(Product_id)s, %(Product_name)s, %(Product_platform)s, %(Product_price)s, %(Time)s)]
[parameters: ({'Product_name': 'Samsung 32 litres Convection Microwave Oven (MC32K7056CB/TL, Black)', 'Date': '2020-07-04', 'Product_discount': '13%', 'Product_price': '25,850.00', 'Product_platform': 'Croma', 'Time': '01:14:07', 'Product_Category': 'MW', 'Product_discounted_price': '22,499.00', 'Product_id': 'MC32K7056CB/TL', 'Product_availability': 'Nan'}, {'Product_name': 'Samsung 81 cm (32 inch) HD LED Smart TV (32N4200, Black)', 'Date': '2020-07-04', 'Product_discount': '28